In [1]:
import glob
import math
import pickle
from stopwords import stops

In [5]:

smooth_poswordprobs = {}
smooth_negwordprobs = {}

In [20]:
def read_in_training_data():
    positivewords = []
    allpos = glob.glob("pos/*")
    for filename in allpos:
        f = open(filename)
        thesewords = set()
        for line in f:
            words = line.rstrip().split()
            for w in words:
                if w not in stops:
                    thesewords.add(w)
        f.close()
        positivewords.extend(list(thesewords))
    
#     print(len(positivewords), "positive tokens found!")
#     print(len(set(positivewords)), "positive types found!")
    
    negativewords = []
    allneg = glob.glob("neg/*")
    for filename in allneg:
        f = open(filename)
        thesewords = set()
        for line in f:
            words = line.rstrip().split()
            for w in words:
                if w not in stops:
                    thesewords.add(w)
        f.close()
        negativewords.extend(list(thesewords))    
#     print(len(negativewords), "negative tokens found!")
#     print(len(set(negativewords)), "negative types found!")

    pickle.dump(positivewords,open('pickleData/poswords.pickle','wb'))
    pickle.dump(negativewords,open('pickleData/negwords.pickle','wb'))
    
    
    return


read_in_training_data()

In [17]:
def calculate_smooth_nb_probabilities():

    poswords = pickle.load(open('pickleData/poswords.pickle','rb'))
    negwords = pickle.load(open('pickleData/negwords.pickle','rb'))
    
    
    posFdist = FreqDist(poswords)
    negFdist = FreqDist(negwords)
    posTypes = posFdist.most_common()
    negTypes = negFdist.most_common()
    
    # +1 smoothing to can handle unseen words.
    for word in posTypes:
        smooth_poswordprobs[word[0]] = math.log((word[1]+1)/(len(poswords)+len(posTypes)+ 1))
    for word in negTypes:
        smooth_negwordprobs[word[0]] = math.log((word[1]+1)/(len(negwords)+len(negTypes)+ 1))
     
    pickle.dump(smooth_poswordprobs,open('pickleData/poswordprobs.pickle','wb'))
    pickle.dump(smooth_negwordprobs,open('pickleData/negwordprobs.pickle','wb'))

    return

calculate_smooth_nb_probabilities()


In [18]:
def smooth_naive_bayes(reviewwords):

    posscore = 0
    negscore = 0
    
    poswords = pickle.load(open('pickleData/poswords.pickle','rb'))
    negwords = pickle.load(open('pickleData/negwords.pickle','rb'))
    
    posdefaultprob = math.log(1/(len(poswords)+len(set(poswords))+ 1))
    negdefaultprob = math.log(1/(len(negwords)+len(set(negwords))+ 1))
    
    smooth_poswordprobs = pickle.load(open('pickleData/poswordprobs.pickle','rb'))
    smooth_negwordprobs = pickle.load(open('pickleData/negwordprobs.pickle','rb'))

    posscore = smooth_poswordprobs.get(reviewwords[0], posdefaultprob)
    for i in range(1, len(reviewwords)):
        posscore += smooth_poswordprobs.get(reviewwords[i], posdefaultprob)


    negscore = smooth_negwordprobs.get(reviewwords[0], negdefaultprob)
    for i in range(1, len(reviewwords)):
        negscore += smooth_negwordprobs.get(reviewwords[i], negdefaultprob)

   
    if (posscore - negscore) >  0:
        return "pos"

    return "neg"

In [19]:
import re
def calculate_accuracy():
    smnbcorrect = 0

    # read in the test reviews
    testdata = glob.glob("test/*")
    for filename in testdata:
        wholereview = ""
        reviewwords = []
        with open(filename, encoding='utf8') as f:
            wholereview = f.read().rstrip()
        words = set(wholereview.split())
        for w in words:
            if w not in stops:
                reviewwords.append(w)
            
        filepolarity = re.sub(r"^.*?(pos|neg)-.*?$", r"\1", filename)

        if filepolarity == smooth_naive_bayes(reviewwords):
            smnbcorrect += 1
     

    print("accuracy: ", (smnbcorrect/200))
calculate_accuracy()

accuracy:  0.845
